# Deep Neural Network for MNIST Classification

We'll apply all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs). 

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 

Our goal would be to build a neural network with 2 hidden layers.

## Import the relevant packages

In [1]:
import numpy as np
import tensorflow as tf

# TensorFLow includes a data provider for MNIST that we'll use.
# It comes with the tensorflow-datasets module, therefore, if you haven't please install the package using
# pip install tensorflow-datasets 
# or
# conda install tensorflow-datasets

import tensorflow_datasets as tfds

# these datasets will be stored in C:\Users\*USERNAME*\tensorflow_datasets\...
# the first time you download a dataset, it is stored in the respective folder 
# every other time, it is automatically loading the copy on your computer 

## Data

That's where we load and preprocess our data.

In [2]:
# remember the comment from above
# these datasets will be stored in C:\Users\*USERNAME*\tensorflow_datasets\...
# the first time you download a dataset, it is stored in the respective folder 
# every other time, it is automatically loading the copy on your computer 

# tfds.load actually loads a dataset (or downloads and then loads if that's the first time you use it) 
# in our case, we are interesteed in the MNIST; the name of the dataset is the only mandatory argument
# there are other arguments we can specify, which we can find useful
# mnist_dataset = tfds.load(name='mnist', as_supervised=True)
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
# with_info=True will also provide us with a tuple containing information about the version, features, number of samples
# we will use this information a bit below and we will store it in mnist_info

# as_supervised=True will load the dataset in a 2-tuple structure (input, target) 
# alternatively, as_supervised=False, would return a dictionary
# obviously we prefer to have our inputs and targets separated 
mnist_train,mnist_test=mnist_dataset['train'],mnist_dataset['test']

num_validation_samples=0.1* mnist_info.splits['train'].num_examples
num_validation_samples=tf.cast(num_validation_samples,tf.int64)

num_test_samples=mnist_info.splits['test'].num_examples
num_test_samples=tf.cast(num_test_samples,tf.int64)


# Preprocessing

In [3]:
def scale(image,label):
    image=tf.cast(image,tf.float32)
    image/=255
    return image,label

The above scale function coverts the image numbers (0-255) into number between 0-1 depending upon color intensity. This is a preprocessing step.
The same function can be done by the tf.map function aswell.

In [4]:
scaled_train_and_validation_data=mnist_train.map(scale)
scaled_test_data=mnist_test.map(scale)

In [5]:
buffer_size=10000
shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(buffer_size)

In [6]:
validation_data=shuffled_train_and_validation_data.take(num_validation_samples)

In [7]:
train_data=shuffled_train_and_validation_data.skip(num_validation_samples)

In [8]:
batch_size=100
train_data=train_data.batch(batch_size)
validation_data=validation_data.batch(num_validation_samples)
test_data=scaled_test_data.batch(num_test_samples)

validation_inputs,validation_targets=next(iter(validation_data))

# Model

In [9]:
input_size=784
output_size=10
hidden_layer_size=50

model=tf.keras.Sequential([tf.keras.layers.Flatten(input_shape=(28,28,1)),
                          tf.keras.layers.Dense(hidden_layer_size,activation='relu')
                          ,tf.keras.layers.Dense(hidden_layer_size,activation='relu')
                          ,tf.keras.layers.Dense(output_size,activation='softmax')])

# Loss and Optimizer

In [10]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Training

In [12]:
num_epochs=5
model.fit(train_data,epochs=num_epochs,validation_data=(validation_inputs,validation_targets),validation_steps=5,verbose=2)

Epoch 1/5
540/540 - 51s - loss: 0.0854 - accuracy: 0.9747 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
540/540 - 53s - loss: 0.0752 - accuracy: 0.9769 - val_loss: 0.0813 - val_accuracy: 0.9750
Epoch 3/5
540/540 - 53s - loss: 0.0658 - accuracy: 0.9800 - val_loss: 0.0714 - val_accuracy: 0.9778
Epoch 4/5
540/540 - 52s - loss: 0.0587 - accuracy: 0.9819 - val_loss: 0.0641 - val_accuracy: 0.9810
Epoch 5/5
540/540 - 53s - loss: 0.0513 - accuracy: 0.9846 - val_loss: 0.0746 - val_accuracy: 0.9787


# Testing

In [16]:
test_loss,test_accuracy=model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}'.format(test_loss,test_accuracy*100))

1/1 [==============================] - 6s 6s/step - loss: 0.1113 - accuracy: 0.9684
Test loss: 0.11. Test accuracy: 96.84
